In [8]:
import rust_machine_learning
import pandas as pd
from syed_ml_lib import *

In [9]:
# Get the csv data
df = pd.read_csv("../data/iris.data")
# df = pd.read_csv("../data/kmeans-dataset.csv")
# df = pd.read_csv("../data/winequality-red.csv")
# df = pd.read_csv("../data/digit-recognizer/train.csv")
# df = df.sample(n=10000)
df  

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [10]:
to_encode = [4] # We won't encode the label column. The output layer will use 0-n (assuming classifications are ordered starting at 0), to determine error.
encoded, feature_to_encoded_cols = one_hot_encode(df, to_encode)
encoded

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),is_ class_Iris-setosa,is_ class_Iris-versicolor,is_ class_Iris-virginica
0,5.1,3.5,1.4,0.2,1.0,0.0,0.0
1,4.9,3.0,1.4,0.2,1.0,0.0,0.0
2,4.7,3.2,1.3,0.2,1.0,0.0,0.0
3,4.6,3.1,1.5,0.2,1.0,0.0,0.0
4,5.0,3.6,1.4,0.2,1.0,0.0,0.0
...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0.0,0.0,1.0
146,6.3,2.5,5.0,1.9,0.0,0.0,1.0
147,6.5,3.0,5.2,2.0,0.0,0.0,1.0
148,6.2,3.4,5.4,2.3,0.0,0.0,1.0


In [11]:
# Remove unnecessary data or NAN
to_remove = []
encoded = encoded.drop(encoded.columns[to_remove], axis=1)
encoded

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),is_ class_Iris-setosa,is_ class_Iris-versicolor,is_ class_Iris-virginica
0,5.1,3.5,1.4,0.2,1.0,0.0,0.0
1,4.9,3.0,1.4,0.2,1.0,0.0,0.0
2,4.7,3.2,1.3,0.2,1.0,0.0,0.0
3,4.6,3.1,1.5,0.2,1.0,0.0,0.0
4,5.0,3.6,1.4,0.2,1.0,0.0,0.0
...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0.0,0.0,1.0
146,6.3,2.5,5.0,1.9,0.0,0.0,1.0
147,6.5,3.0,5.2,2.0,0.0,0.0,1.0
148,6.2,3.4,5.4,2.3,0.0,0.0,1.0


In [12]:
# Prepare data for training with input rust functions expect
label_col = 2

# Normalize columns
# labels = encoded.iloc[:, label_col]
# max_per_col = encoded.abs().max().replace(0,1)
# encoded = (encoded / max_per_col)
# encoded.iloc[:, label_col] = labels

train_data, test_data = split_train_test(encoded, 0.9, True)

# Remove any encoded data of the same class
train_data_features = train_data.copy()
remove_encoded_category_data_and_label(train_data_features, feature_to_encoded_cols, label_col)

train_data_features_list = train_data_features.values.tolist()

# Data Correlation
corr = train_data_features.corr()
fig = px.imshow(corr, aspect="auto", origin='lower')
fig.show()

# Get labels
train_data_labels = train_data.iloc[:, label_col].values.tolist()
train_data_features

,sepal length (cm),sepal width (cm),petal width (cm),is_ class_Iris-setosa,is_ class_Iris-versicolor,is_ class_Iris-virginica
56,6.3,3.3,1.6,0.0,1.0,0.0
17,5.1,3.5,0.3,1.0,0.0,0.0
87,6.3,2.3,1.3,0.0,1.0,0.0
79,5.7,2.6,1.0,0.0,1.0,0.0
127,6.1,3.0,1.8,0.0,0.0,1.0
...,...,...,...,...,...,...
32,5.2,4.1,0.1,1.0,0.0,0.0
22,4.6,3.6,0.2,1.0,0.0,0.0
72,6.3,2.5,1.5,0.0,1.0,0.0
28,5.2,3.4,0.2,1.0,0.0,0.0


In [13]:
# Now train the model
hidden_layer_sizes = [20,20]
num_features = len(train_data_features_list[0])
num_classifications = 1 # Tells the library to switch to regression mode
learning_rate = 1e-4
num_iterations = 20000
batch_size = 0
train_data_features_list
trained_model = rust_machine_learning.BasicNeuralNetwork(hidden_layer_sizes, num_features, num_classifications)
trained_model.train(train_data_features_list, train_data_labels, learning_rate, num_iterations, batch_size)

% Correct: 0
% Correct: 34.814816
% Correct: 29.62963
% Correct: 20.74074
% Correct: 21.481482
% Correct: 17.037037
% Correct: 14.074074
% Correct: 16.296297
% Correct: 15.555555
% Correct: 17.037037
% Correct: 16.296297
% Correct: 17.037037
% Correct: 17.037037
% Correct: 19.25926
% Correct: 20
% Correct: 18.518518
% Correct: 18.518518
% Correct: 17.777779
% Correct: 17.777779
% Correct: 17.777779
% Correct: 16.296297
% Correct: 20.74074
% Correct: 19.25926
% Correct: 20
% Correct: 23.703703
% Correct: 25.185184
% Correct: 30.37037
% Correct: 26.666666
% Correct: 26.666666
% Correct: 30.37037
% Correct: 31.851852
% Correct: 32.592594
% Correct: 33.333332
% Correct: 31.11111
% Correct: 34.074074
% Correct: 36.296295
% Correct: 34.074074
% Correct: 36.296295
% Correct: 36.296295
% Correct: 37.037037
% Correct: 35.555557
% Correct: 34.074074
% Correct: 35.555557
% Correct: 37.037037
% Correct: 34.814816
% Correct: 40
% Correct: 37.037037
% Correct: 37.77778
% Correct: 40
% Correct: 39.25

In [14]:
# Now let us test the model
test_data_features = test_data.copy()
remove_encoded_category_data_and_label(test_data_features, feature_to_encoded_cols, label_col)
test_data_features_list = test_data_features.values.tolist()
test_data_labels = test_data[test_data.columns[label_col]].values.tolist()

labelled_results = trained_model.regression(test_data_features_list)

num_correct = 0
tolerance = 0.05
for test_label, result_label in zip(test_data_labels, labelled_results):
  num_correct += (abs(test_label-result_label)/(1.0 if test_label == 0 else test_label) <= tolerance)
print("Percent Correct: ", 100.0 * num_correct / len(test_data_labels))
labelled_results

Percent Correct:  40.0


[1.4186654090881348,
 1.288148045539856,
 1.5077838897705078,
 5.485699653625488,
 1.4600642919540405,
 1.4663852453231812,
 1.8578524589538574,
 1.478707194328308,
 4.859255790710449,
 6.1548333168029785,
 1.3073433637619019,
 4.386623382568359,
 3.926624059677124,
 4.390395641326904,
 6.202131271362305]